# Aula 1 - Criando um agente de funções

## Vídeo 1.2 - Configurando o LlamaIndex

In [ ]:
!pip install llama-index llama_index.embeddings.huggingface llama-index-readers-file llama-index-llms-groq arxiv llama-index-tools-tavily-research crewai crewai-tools gradio

In [ ]:
from llama_index.core import Settings

In [ ]:
from google.colab import userdata
groq = userdata.get('GROQ_API_KEY')

In [ ]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-groq-70b-8192-tool-use-preview", api_key=groq)

In [ ]:
Settings.llm = llm

In [ ]:
def calcular_engajamento(curtidas: int, comentarios: int, compartilhamentos: int, seguidores: int) -> str:
    """
    Calcula o engajamento total e a taxa de engajamento de uma postagem.

    Args:
        curtidas (int): Número de curtidas na postagem.
        comentarios (int): Número de comentários na postagem.
        compartilhamentos (int): Número de compartilhamentos da postagem.
        seguidores (int): Número total de seguidores.

    Returns:
        dict: Um dicionário contendo o engajamento total (int) e a taxa de engajamento (float, em porcentagem).
    """
    engajamento_total = curtidas + comentarios + compartilhamentos
    taxa_engajamento = (engajamento_total / seguidores) * 100 if seguidores > 0 else 0
    dicionario = {"engajamento_total": engajamento_total,"taxa_engajamento": round(taxa_engajamento, 2)}
    resultado = f"O engajamento total é {engajamento_total} e a taxa de engajamento é {round(taxa_engajamento, 2)}%."
    # Return the dictionary directly
    return resultado

## Vídeo 1.3 - Transformando a função em ferramenta

In [ ]:
from llama_index.core.tools import FunctionTool

In [ ]:
ferramenta = FunctionTool.from_defaults(fn=calcular_engajamento,name="Calcular Engajamento",description=(
        "Calcula o engajamento total e a taxa de engajamento de uma postagem nas redes sociais. "
        "Argumentos: curtidas (int), comentarios (int), compartilhamentos (int), seguidores (int). "
        "Retorna uma string com o engajamento total e a taxa de engajamento."
    ))

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm,
)

In [ ]:
from llama_index.core.agent import AgentRunner

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response =agent.chat("Qual é o engajamento de uma postagem que teve 150 curtidas, 35 comentários, 20 compartilhamentos, e o perfil tem 2000 seguidores?")

In [ ]:
response = agent.chat("Quem é Albert Einstein?")

## Vídeo 1.4 - Consultando artigos

In [ ]:
import arxiv

In [ ]:
def consulta_artigos(titulo: str) -> str:
    """
    Consulta os artigos na base de dados arXiv e retorna resultados formatados.
    """
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = []
    for resultado in busca.results():
        resultados.append(f"Título: {resultado.title}\n"
                          f"Resumo: {resultado.summary}\n"
                          f"Categoria: {resultado.primary_category}\n"
                          f"Link: {resultado.entry_id}\n")

    return "\n\n".join(resultados)

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("Me retorne artigos sobre o uso da inteligência artificial nas redes sociais")